In [1]:
import numpy as np
import pandas as pd
import copy

df = pd.read_csv("groceries.csv")
data = []
for row in range(0,9834):
    temp = []
    for item in df.iloc[row]:
        if(item == item):
            temp.append(item)
    data.append(temp)        

In [2]:
supportThreshold = 1.0
confiThreshold = .5

In [3]:
def support(data,element):
    count = 0;
    for row in data:
        flag = 0
        item = list(element)
        for item in element:
            if item not in row:
                flag = 1
        if flag == 0:
            count+=1
    return count * 100/len(data)

In [4]:
def unique_items(data):
    temp = set()
    for row in data:
        for item in row:
            temp.add(item)
    return temp;

In [5]:
def initial_items(unq):
    strt = []
    for item in unq:
        item = [item]
        if support(data,item) >= supportThreshold:
            item = set(item)
            strt.append(item)
    return strt

In [6]:
def separate(itemset):
    _ = set()
    for i in itemset:
        for item in i:
            _.add(item)
    return list(_)

In [7]:
def unique_list(L):
    return [set(item) for item in set(frozenset(item) for item in L)]

def joinSet(itemSet):
    itemList = separate(itemSet)
    result = []
    for Set in itemSet:
        for item in itemList:
            temp = copy.copy(Set)
            if item in temp:
                continue
            temp.add(item)
            if temp in result:
                continue
            result.append(temp)
    return result

In [8]:
def prune(C, L):
    res = []
    for Set in C:
        ok = True
        for item in Set:
            if ok == False:
                break
            _set = copy.copy(Set)
            _set.remove(item)
            if _set not in L:
                ok = False
        if ok:
            res.append(Set)
    return res

In [9]:
def SetToStr(S):
    _str = ''
    for item in S:
        _str += item
        _str += ','
    _str = _str[:-1]
    return '{' + _str + '}' + '[' + str(round(0.01 * len(data) * support(data, S))) + ']'

def convert(S, T):
    return SetToStr(S) + ' => ' + SetToStr(T)

In [10]:
def getConfidences(confidences, Set):
    
    _set = list(Set)
    n = len(_set)
    invalid = set()
    
    for mask in range(1 << n):
        _tmp = set()
        _tmp_ = set()
        
        ok = True
        for submask in invalid:
            if mask&submask == mask:
                ok = False
        if not ok:
            continue
        
        for i in range(n):
            if (1 << i) & mask:
                _tmp_.add(_set[i])
            else:
                _tmp.add(_set[i])

        if len(_tmp) == 0 or len(_tmp_) == 0:
            continue
        confi = support(data, Set) / support(data, _tmp)

        if (confi < confiThreshold):
            invalid.add(mask)
        else:
            confidences[convert(_tmp, _tmp_)] = confi

In [11]:
def Apriori(confidences, start, file):
    L = start[:]
    C = []
    C = joinSet(L)
    C = prune(C, L)
    Lnext = []
    for _set in C:
        if support(data,list(_set)) >= supportThreshold:
            Lnext.append(_set)
    if len(Lnext) == 0:
        return L
    else:
        print(len(Lnext))
        for Set in Lnext:
            file.write(SetToStr(Set))
            file.write('\n')
            getConfidences(confidences, Set)
        return Apriori(confidences, Lnext, file)

In [12]:
uni = unique_items(data)

In [13]:
init = initial_items(uni)

In [14]:
confidences = dict()
file = open('freq_itemset_sup=1_conf=5e-1.txt', 'w')
res = Apriori(confidences, init, file)
file.close()

213
32


In [15]:
with open('assn_rules_sup=1_conf=5e-1.txt', 'w') as file:
    for a in confidences:
        file.write(a)
        file.write('  || confidence = ' + str(confidences[a]))
        file.write('\n')